In [153]:
import requests
import time
import re
import pandas as pd
import pprint
import pycrfsuite

from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_xy
from bs4 import BeautifulSoup
from konlpy.tag import Kkma as kkm
from konlpy.tag import Okt
from konlpy.tag import Mecab

In [55]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [188]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [189]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1=dict(ind_reviews(1001))

In [190]:
reviews1

{'태백에 있다는 위치적인 단점은 있지만 일을 배우기에 좋은 병원': '70',
 '뚜렷한 목표의식이 있다면 좋은 경험과 경력이 될 수 있는 곳!': '70',
 '월급 중요하지 않고 오로지 정년 보장만 중요하다면 공무원 추천함': '70',
 '일단 파벌싸움이심하다 진짜심각하다 매우심각하다 \r그러나 직원복지는 작살난다': '70',
 '주먹구구식에다 인수인계 개판임, 워라밸 보장도 안됨': '70',
 '지방 공공기관이지만 수직적이지 않음. 신입도 일하기 편함. 워라벨 좋음.': '70',
 '서민금융생활법에 따라 9월 특별 법인화가 확정된 채무조정기구- 채무 감면 및 소액금융등이 주 업무': '70',
 '기술뿐만이 아니라 정책적인 부분까지 모두 도맡아 할 수 있는 곳': '70',
 '- 고시/비고시 출신 가림- 행정직이 일하기 좋음 그 외 직렬은 X- 업무량 많음- 조직이 커서 여러 업무 배우기 좋음': '70',
 '재외공관 대표부에서 계약직으로 근무. 태극마크라는 자부심으로 일하는 조직': '70',
 '서울시립 청소년수련시설로 청소년활동 프로그램운영 및 지원, 평생교육프로그램 운영, 천문과학특화 프로그램 운영 등을 주요 사업으로 하고 있으며, 직원간의 유대 및 협력관계가 좋음': '38',
 '기관장 말에 이리저리 휘둘리고 싶으면 입사하세요 기관장 말이 곧 법인 곳입니다(올해 리뷰들 조작아닌가 의심될 정도로 황당..)': '38',
 '울산문화재단 CCTV 불법사찰, 감시에 대해 정부는 특검을 실시하라! 건전한 조직문화 해치는 불법행위 근절하라!! <cctv 불법사찰 규탄 운동>': '38',
 '체계가 없는 회사이고 나름대로 분위기는 수평적인것같습니다': '38',
 '끊임없는 삽질과 고성 그리고 독재와 이기주의가 팽배한 행복한 우리 회사': '38',
 '안정적인 공공기관. 퇴사한지 조금 되었는데 그때는 괜찮았습니다.': '38',
 '근본없는 회사라 어떤일이 일어나도 납득이 됨.조례도 개판이고 뭐 제대로 갖춰진게 하나도 없음.': '38',

In [191]:
text1=list(reviews1.keys())
score1=list(reviews1.values())

In [213]:
sum=0
for i in range(len(score1)):
    sum+=int(score1[i])
avr=sum/len(score1)
print("score 평균값: ", round(avr,3))

score 평균값:  64.133


In [134]:
sent_num=0
text1_sent=[]
for i in range(len(text1)):
    text1_pos=okt.pos(text1[i])
    sent_num+=1
    for t in text1_pos:
        n=list(t)
        n.insert(0, sent_num)
        n.insert(3, score1[(sent_num-1)])
        text1_sent.append(n)

pprint.pprint(text1_sent)

[[1, '위치', 'Noun', '52'],
 [1, '가', 'Josa', '52'],
 [1, '좋음', 'Adjective', '52'],
 [1, ',', 'Punctuation', '52'],
 [1, '연봉', 'Noun', '52'],
 [1, '나쁘지', 'Adjective', '52'],
 [1, '않음', 'Verb', '52'],
 [1, ',', 'Punctuation', '52'],
 [1, '부바', 'Noun', '52'],
 [1, '부', 'Noun', '52'],
 [1, ',', 'Punctuation', '52'],
 [1, '이상한', 'Adjective', '52'],
 [1, '곳', 'Noun', '52'],
 [1, '은', 'Josa', '52'],
 [1, '너무', 'Adverb', '52'],
 [1, '이상함', 'Adjective', '52'],
 [1, '좋은', 'Adjective', '52'],
 [1, '부서', 'Noun', '52'],
 [1, '는', 'Josa', '52'],
 [1, '좋음', 'Adjective', '52'],
 [2, '공공기관', 'Noun', '52'],
 [2, '이라서', 'Josa', '52'],
 [2, '복지', 'Noun', '52'],
 [2, '혜택', 'Noun', '52'],
 [2, '사', 'Modifier', '52'],
 [2, '기업', 'Noun', '52'],
 [2, '보다', 'Josa', '52'],
 [2, '좋음', 'Adjective', '52'],
 [2, '안정', 'Noun', '52'],
 [2, '적', 'Suffix', '52'],
 [2, '인', 'Josa', '52'],
 [2, '직장', 'Noun', '52'],
 [3, '1년', 'Number', '52'],
 [3, '뒤', 'Noun', '52'],
 [3, '인원', 'Noun', '52'],
 [3, '의', 'Josa', '52'],
 [3, 

 [75, '안정', 'Noun', '74'],
 [75, '적', 'Suffix', '74'],
 [75, '이고', 'Josa', '74'],
 [75, '마음', 'Noun', '74'],
 [75, '편하게', 'Adjective', '74'],
 [75, '일', 'Noun', '74'],
 [75, '할', 'Verb', '74'],
 [75, '수', 'Noun', '74'],
 [75, '있는', 'Adjective', '74'],
 [75, '직장', 'Noun', '74'],
 [76, '생활', 'Noun', '74'],
 [76, '체육', 'Noun', '74'],
 [76, '을', 'Josa', '74'],
 [76, '중점', 'Noun', '74'],
 [76, '적', 'Suffix', '74'],
 [76, '으로', 'Josa', '74'],
 [76, '관리', 'Noun', '74'],
 [76, '하고', 'Josa', '74'],
 [76, '관련', 'Noun', '74'],
 [76, '정책', 'Noun', '74'],
 [76, '을', 'Josa', '74'],
 [76, '다룰', 'Verb', '74'],
 [76, '수', 'Noun', '74'],
 [76, '있는', 'Adjective', '74'],
 [76, '곳', 'Noun', '74'],
 [77, '농업', 'Noun', '74'],
 [77, '직', 'Noun', '74'],
 [77, '으로', 'Josa', '74'],
 [77, '경력', 'Noun', '74'],
 [77, '쌓기에', 'Verb', '74'],
 [77, '괜찮으며', 'Adjective', '74'],
 [77, '의지', 'Noun', '74'],
 [77, '만', 'Josa', '74'],
 [77, '있다면', 'Adjective', '74'],
 [77, '많이', 'Adverb', '74'],
 [77, '배울', 'Verb', '74'],
 [7

 [133, '나', 'Josa', '60'],
 [133, '민원', 'Noun', '60'],
 [133, '인들', 'Josa', '60'],
 [133, '이', 'Determiner', '60'],
 [133, '조금', 'Noun', '60'],
 [133, '견디기', 'Verb', '60'],
 [133, '힘든', 'Adjective', '60'],
 [133, '곳', 'Noun', '60'],
 [133, '.', 'Punctuation', '60'],
 [134, '사회', 'Noun', '60'],
 [134, '복지', 'Noun', '60'],
 [134, '전공자', 'Noun', '60'],
 [134, '들', 'Suffix', '60'],
 [134, '은', 'Josa', '60'],
 [134, '한', 'Verb', '60'],
 [134, '번', 'Noun', '60'],
 [134, '쯤', 'Suffix', '60'],
 [134, '경험', 'Noun', '60'],
 [134, '해봐도', 'Verb', '60'],
 [134, '괜찮은', 'Adjective', '60'],
 [134, '직장', 'Noun', '60'],
 [135, '해외', 'Noun', '60'],
 [135, '주재', 'Noun', '60'],
 [135, '공공기관', 'Noun', '60'],
 [135, '이', 'Josa', '60'],
 [135, '어떻게', 'Adjective', '60'],
 [135, '돌아가는지', 'Verb', '60'],
 [135, '경험', 'Noun', '60'],
 [135, '해볼', 'Verb', '60'],
 [135, '수', 'Noun', '60'],
 [135, '있다', 'Adjective', '60'],
 [136, '가족', 'Noun', '60'],
 [136, '적', 'Suffix', '60'],
 [136, '인', 'Josa', '60'],
 [136, '분위기'

 [200, '10', 'Number', '58'],
 [200, '개월', 'Noun', '58'],
 [200, '계약', 'Noun', '58'],
 [200, '직', 'Noun', '58'],
 [200, '다니고', 'Verb', '58'],
 [200, '실업', 'Noun', '58'],
 [200, '급', 'Suffix', '58'],
 [200, '여', 'Josa', '58'],
 [200, '받고', 'Verb', '58'],
 [200, '그러다가', 'Conjunction', '58'],
 [200, '끝', 'Noun', '58'],
 [200, '정규직', 'Noun', '58'],
 [200, '은', 'Josa', '58'],
 [200, '안되고', 'Adjective', '58'],
 [200, '그냥', 'Noun', '58'],
 [200, '10', 'Number', '58'],
 [200, '개월', 'Noun', '58'],
 [200, '잘', 'VerbPrefix', '58'],
 [200, '다님', 'Verb', '58'],
 [201, '환승', 'Noun', '56'],
 [201, '하기에', 'Verb', '56'],
 [201, '좋은', 'Adjective', '56'],
 [201, '기업', 'Noun', '56'],
 [201, '.', 'Punctuation', '56'],
 [201, '경력', 'Noun', '56'],
 [201, '한', 'Determiner', '56'],
 [201, '줄', 'Noun', '56'],
 [201, '남겨', 'Verb', '56'],
 [201, '좋았고', 'Adjective', '56'],
 [201, '앞', 'Noun', '56'],
 [201, '으로는', 'Josa', '56'],
 [201, '보지말자', 'Verb', '56'],
 [202, '서류', 'Noun', '56'],
 [202, '작업', 'Noun', '56'],
 

 [260, '.', 'Punctuation', '66'],
 [261, '직원', 'Noun', '66'],
 [261, '들', 'Suffix', '66'],
 [261, '과의', 'Josa', '66'],
 [261, '사이', 'Noun', '66'],
 [261, '가', 'Josa', '66'],
 [261, '돈득', 'Noun', '66'],
 [261, '하고', 'Josa', '66'],
 [261, '기업', 'Noun', '66'],
 [261, '분위기', 'Noun', '66'],
 [261, '가', 'Josa', '66'],
 [261, '좋은', 'Adjective', '66'],
 [261, '면도', 'Noun', '66'],
 [261, '있음', 'Adjective', '66'],
 [262, '이름', 'Noun', '66'],
 [262, '이', 'Josa', '66'],
 [262, '유명한', 'Adjective', '66'],
 [262, '만큼', 'Noun', '66'],
 [262, '좋은', 'Adjective', '66'],
 [262, '기업', 'Noun', '66'],
 [262, '.', 'Punctuation', '66'],
 [262, '공', 'Modifier', '66'],
 [262, '기업인', 'Noun', '66'],
 [262, '만큼', 'Josa', '66'],
 [262, '편하다', 'Adjective', '66'],
 [262, '.', 'Punctuation', '66'],
 [262, '자기계발', 'Noun', '66'],
 [262, '을', 'Josa', '66'],
 [262, '할', 'Verb', '66'],
 [262, '수', 'Noun', '66'],
 [262, '있는', 'Adjective', '66'],
 [262, '시간', 'Noun', '66'],
 [262, '이', 'Josa', '66'],
 [262, '주어진다고', 'Verb', '

In [170]:
text1_sent

[[1, '위치', 'Noun', '52'],
 [1, '가', 'Josa', '52'],
 [1, '좋음', 'Adjective', '52'],
 [1, ',', 'Punctuation', '52'],
 [1, '연봉', 'Noun', '52'],
 [1, '나쁘지', 'Adjective', '52'],
 [1, '않음', 'Verb', '52'],
 [1, ',', 'Punctuation', '52'],
 [1, '부바', 'Noun', '52'],
 [1, '부', 'Noun', '52'],
 [1, ',', 'Punctuation', '52'],
 [1, '이상한', 'Adjective', '52'],
 [1, '곳', 'Noun', '52'],
 [1, '은', 'Josa', '52'],
 [1, '너무', 'Adverb', '52'],
 [1, '이상함', 'Adjective', '52'],
 [1, '좋은', 'Adjective', '52'],
 [1, '부서', 'Noun', '52'],
 [1, '는', 'Josa', '52'],
 [1, '좋음', 'Adjective', '52'],
 [2, '공공기관', 'Noun', '52'],
 [2, '이라서', 'Josa', '52'],
 [2, '복지', 'Noun', '52'],
 [2, '혜택', 'Noun', '52'],
 [2, '사', 'Modifier', '52'],
 [2, '기업', 'Noun', '52'],
 [2, '보다', 'Josa', '52'],
 [2, '좋음', 'Adjective', '52'],
 [2, '안정', 'Noun', '52'],
 [2, '적', 'Suffix', '52'],
 [2, '인', 'Josa', '52'],
 [2, '직장', 'Noun', '52'],
 [3, '1년', 'Number', '52'],
 [3, '뒤', 'Noun', '52'],
 [3, '인원', 'Noun', '52'],
 [3, '의', 'Josa', '52'],
 [3, 

In [75]:
dict_pos=[]
dict_neg=[]
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/pos_pol_word.txt", 'rt', encoding='UTF8'):
    dict_pos.append(line.split('\n')[0])
    
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/neg_pol_word.txt", 'rt', encoding='UTF8'):
    dict_neg.append(line.split('\n')[0])

In [76]:
dict_pos

['출처',
 '1) 표준국어대사전 뜻풀이',
 '-> https://github.com/mrchypark/stdkor',
 '2) 신조어',
 '-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D',
 '3) 이모티콘',
 '-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98',
 '4) SentiWordNet_3.0.0_20130122',
 '-> http://sentiwordnet.isti.cnr.it/',
 '5) SenticNet-5.0',
 '-> http://sentic.net/',
 '6) 감정단어사전0603',
 '-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 ',
 '7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원',
 '',
 '',
 '단어 수 : 4863',
 '극성 : 긍정',
 '',
 '(-;',
 '(^^)',
 '(^-^)',
 '(^^*',
 '(^_^)',
 '(^o^)',
 '*^^*',
 '/^o^\\',
 ':(',
 ":'-(",
 ':-(',
 ':-)',
 ':|',
 ';',
 ';)',
 ';-)',
 '^///^',
 '^^',
 '^^//',
 '<3',
 '=)',
 'XD',
 'ㄱㅅ',
 'ㄱㅇㄷ',
 '가격이 싸다',
 '가까이 사귀어',
 '가까이하다',
 '가다듬어',
 '가다듬어 수습하는',
 '가다듬어 수습하다',
 '가다듬어 정하다',
 '가당히',
 '가뜬하게',
 '가라앉혀 바로잡다',
 '가르치며',
 '가르침',
 '가벼운 마음으로',
 '가볍게 웃

In [77]:
dict_neg

['출처',
 '1) 표준국어대사전 뜻풀이',
 '-> https://github.com/mrchypark/stdkor',
 '2) 신조어',
 '-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D',
 '3) 이모티콘',
 '-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98',
 '4) SentiWordNet_3.0.0_20130122',
 '-> http://sentiwordnet.isti.cnr.it/',
 '5) SenticNet-5.0',
 '-> http://sentic.net/',
 '6) 감정단어사전0603',
 '-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 ',
 '7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원',
 '',
 '',
 '단어 수 : 9826',
 '극성 : 부정',
 '',
 '가난',
 '가난뱅이',
 '가난살이',
 '가난살이하다',
 '가난설음',
 '가난에',
 '가난에 쪼들려서',
 '가난하게',
 '가난하고',
 '가난하고 어렵다',
 '가난하고 천한',
 '가난하다',
 '가난하여',
 '가난한',
 '가난한 데다가',
 '가난한 사람을',
 '가난히',
 '가년스럽다',
 '가녕스럽다',
 '가누지 못하고',
 '가누지 못하다',
 '가누지 못할',
 '가누지 못할 정도로',
 '가누지 못함',
 '가눌 수 없게',
 '가눌 수 없을',
 '가눌 수 없이',
 '가당찮다',
 '가당찮이',
 '가두거나 해치거나',
 '가망이 없게',
 '가슴 아파하다',
 '가슴 아파함',

In [171]:
data = pd.DataFrame(text1_sent) # data를 pandas로 변환

In [172]:
data.columns = ['Sentence #', 'Word', 'POS', 'score']

In [173]:
data.head(10)

,Sentence #,Word,POS,score
0,1,위치,Noun,52
1,1,가,Josa,52
2,1,좋음,Adjective,52
3,1,",",Punctuation,52
4,1,연봉,Noun,52
5,1,나쁘지,Adjective,52
6,1,않음,Verb,52
7,1,",",Punctuation,52
8,1,부바,Noun,52
9,1,부,Noun,52


In [174]:
data.tail(10)

,Sentence #,Word,POS,score
5515,300,아직도,Adverb,56
5516,300,이런,Modifier,56
5517,300,곳,Noun,56
5518,300,이,Josa,56
5519,300,있다는,Adjective,56
5520,300,것,Noun,56
5521,300,이,Josa,56
5522,300,신기한,Adjective,56
5523,300,조직,Noun,56
5524,300,.,Punctuation,56


In [193]:
data.isnull().sum() # 비어있는 값 확인 결과: 없음

Sentence #    0
Word          0
POS           0
score         0
dtype: int64

In [195]:
words = list(set(data["Word"].values))
n_words = len(words) # 고유한 단어들이 1784개(문장부호 포함)
n_words

1784

In [196]:
# 데이터에서 문장을 원하는 형태로 포매팅하여 추출하는 클래스
# Word, POS, Tag -> data를 여러 함수로 씌워서 봄. 
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        # for문 사용법 보기. lambda: 일종의 함수(적용하는), data에 다음과 같은 기능 적용
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["score"].values.tolist())]
        # sentence 단위로 grouped
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        # 그룹화된 데이터를 다시 sentences로 저장
        self.sentences = [s for s in self.grouped]
    
    # get_next: 다음 문장 처리
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [197]:
# 전체 문장 추출
getter = SentenceGetter(data)

In [198]:
# 문장 하나
sent = getter.get_next()

In [199]:
# 추출한 문장 확인(적용 결과)
print(sent)

None
